In [20]:
# Импорт необходимых библиотек
import requests  # Для выполнения HTTP-запросов к веб-сайтам
from bs4 import BeautifulSoup  # Для парсинга HTML и извлечения данных
import pandas as pd  # Для работы с табличными данными и создания DataFrame
import time  # Для добавления задержек между запросами
import re  # Для работы с регулярными выражениями
import json  # Для работы с JSON форматом (хотя в этой версии не используется)
from datetime import datetime  # Для работы с датой и временем
import os  # Для работы с файловой системой (создание папок)
from urllib.parse import urljoin  # Для корректного объединения URL

# Создание класса для парсера сайта HomeConcept
class HomeConceptParser:
    # Инициализация класса
    def __init__(self):
        # Заголовки для HTTP-запросов, чтобы сайт воспринимал нас как браузер
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        }
        # Создание сессии для сохранения состояния между запросами
        self.session = requests.Session()
        # Установка заголовков для всех запросов сессии
        self.session.headers.update(self.headers)
        
    # Метод для безопасной загрузки страницы с обработкой ошибок
    def get_page_safe(self, url, max_retries=3):
        """Безопасная загрузка страницы"""
        # Цикл повторных попыток при ошибках
        for attempt in range(max_retries):
            try:
                # Выполнение GET-запроса с таймаутом 15 секунд
                response = self.session.get(url, timeout=15)
                # Проверка статуса ответа (вызовет исключение при ошибке)
                response.raise_for_status()
                # Возврат успешного ответа
                return response
            except requests.exceptions.RequestException as e:
                # Вывод сообщения об ошибке
                print(f"Ошибка при загрузке (попытка {attempt + 1}): {e}")
                # Если остались попытки, ждем перед повторной попыткой
                if attempt < max_retries - 1:
                    time.sleep(2)
                else:
                    # Если попытки закончились, возвращаем None
                    return None
    
    # Метод для преобразования текста цены в числовое значение
    def parse_price(self, price_text):
        """Парсит цену из текста"""
        # Если текст пустой, возвращаем 0
        if not price_text:
            return 0
        try:
            # Удаляем все нецифровые символы из текста цены
            cleaned_price = re.sub(r'[^\d]', '', str(price_text))
            # Преобразуем в число, если есть цифры, иначе возвращаем 0
            return int(cleaned_price) if cleaned_price else 0
        except (ValueError, TypeError):
            # Возвращаем 0 при ошибке преобразования
            return 0
    
    # Основной метод для извлечения данных из карточки товара
    def extract_product_data(self, product_card):
        """Извлекает данные из карточки товара на основе анализа структуры"""
        try:
            # Создаем словарь для хранения данных товара
            product_data = {}
            
            # 1. Поиск названия товара - это текст ссылки без "Распродажа X%"
            # Находим все ссылки в карточке товара
            all_links = product_card.find_all('a')
            # Перебираем все найденные ссылки
            for link in all_links:
                # Получаем текст ссылки и убираем пробелы по краям
                link_text = link.text.strip()
                # Проверяем, что текст есть и это не текст о распродаже
                if link_text and not re.search(r'Распродажа\s*\d+%', link_text):
                    # Сохраняем название товара
                    product_data['title'] = link_text
                    # Сохраняем ссылку на товар
                    if link.get('href'):
                        # Объединяем относительную ссылку с базовым URL
                        product_data['product_url'] = urljoin('https://homeconcept.ru', link.get('href'))
                    # Прерываем цикл после нахождения названия
                    break
            
            # 2. Поиск цены - ищем текст с символом рубля
            price_text = None
            # Получаем весь текст из карточки товара
            all_text = product_card.get_text()
            # Ищем шаблон цены (цифры с пробелами и символ рубля)
            price_match = re.search(r'([\d\s]+)\s*₽', all_text)
            if price_match:
                # Извлекаем найденную цену
                price_text = price_match.group(1)
                # Преобразуем текст цены в число
                product_data['price'] = self.parse_price(price_text)
            else:
                # Если цена не найдена, устанавливаем 0
                product_data['price'] = 0
            
            # 3. Извлечение артикула из URL товара
            if 'product_url' in product_data:
                # Ищем артикул в URL товара с помощью регулярного выражения
                url_match = re.search(r'/product/([^/?]+)', product_data['product_url'])
                if url_match:
                    # Сохраняем найденный артикул
                    product_data['article'] = url_match.group(1)
                else:
                    # Если артикул не найден
                    product_data['article'] = 'N/A'
            else:
                # Если нет ссылки на товар
                product_data['article'] = 'N/A'
            
            # 4. Поиск изображения товара
            img_elem = product_card.find('img')
            if img_elem and img_elem.get('src'):
                # Сохраняем URL изображения
                product_data['image_url'] = urljoin('https://homeconcept.ru', img_elem.get('src'))
            else:
                product_data['image_url'] = 'N/A'
            
            # 5. Проверка наличия скидки
            # Ищем текст о распродаже в карточке товара
            discount_elem = product_card.find(text=re.compile(r'Распродажа\s*\d+%'))
            if discount_elem:
                # Извлекаем процент скидки с помощью регулярного выражения
                discount_match = re.search(r'Распродажа\s*(\d+)%', discount_elem)
                if discount_match:
                    # Сохраняем процент скидки
                    product_data['discount'] = f"{discount_match.group(1)}%"
                else:
                    product_data['discount'] = "Есть скидка"
            else:
                product_data['discount'] = "Нет скидки"
            
            # Добавляем временную метку сбора данных
            product_data['scraped_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # Проверяем, что есть хотя бы название товара
            if 'title' in product_data and product_data['title']:
                # Выводим информацию о найденном товаре
                print(f"✓ Найден товар: {product_data['title'][:50]}... - {product_data['price']} руб")
                return product_data
            else:
                print("✗ Пропускаем товар: не найдено название")
                return None
            
        except Exception as e:
            # Обработка любых ошибок при извлечении данных
            print(f"Ошибка при извлечении данных товара: {e}")
            return None
    
    # Метод для парсинга всей страницы категории
    def parse_category_page(self, url):
        """Парсит страницу категории"""
        # Загружаем страницу
        response = self.get_page_safe(url)
        if not response:
            return []
            
        # Создаем объект BeautifulSoup для парсинга HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        # Список для хранения данных всех товаров
        products_data = []
        
        # Ищем все карточки товаров по классу 'item'
        products = soup.find_all('div', class_='item')
        print(f"Найдено карточек товаров: {len(products)}")
        
        # Обрабатываем каждую карточку товара
        for i, product_card in enumerate(products, 1):
            print(f"Обработка товара {i}/{len(products)}...")
            # Извлекаем данные из карточки
            product_data = self.extract_product_data(product_card)
            if product_data:
                # Добавляем данные в общий список
                products_data.append(product_data)
        
        return products_data
    
    # Метод для очистки и фильтрации данных
    def clean_data(self, df):
        """Очистка данных"""
        if df.empty:
            return df
            
        # Сохраняем начальное количество записей
        initial_count = len(df)
        
        # Удаляем дубликаты по названию и артикулу
        df = df.drop_duplicates(subset=['title', 'article'])
        
        # Удаляем товары без названия
        df = df[df['title'].notna() & (df['title'] != '')]
        
        print(f"После очистки: {len(df)} из {initial_count} товаров")
        return df

# Основная функция программы
def main():
    # Конфигурационные параметры
    CATEGORY_URL = "https://homeconcept.ru/catalog/outdoor/outdoor-sofas/"  # URL категории для парсинга
    OUTPUT_DIR = "homeconcept_data"  # Папка для сохранения результатов
    
    # Создаем директорию для результатов (если не существует)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    # Вывод заголовка программы
    print("=" * 60)
    print("ПАРСЕР HOMECONCEPT - ДИВАНЫ ДЛЯ УЛИЦЫ")
    print("=" * 60)
    
    # Инициализируем парсер
    parser = HomeConceptParser()
    
    # Собираем данные
    print(f"Загрузка страницы: {CATEGORY_URL}")
    all_products = parser.parse_category_page(CATEGORY_URL)
    
    if not all_products:
        print("Не удалось собрать данные!")
        return
    
    print(f"\nУспешно собрано данных: {len(all_products)} товаров")
    
    # Преобразуем список товаров в DataFrame
    df = pd.DataFrame(all_products)
    # Очищаем данные
    df_clean = parser.clean_data(df)
    
    # Генерируем временную метку для имени файла
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if not df_clean.empty:
        # Сохраняем только в Excel (CSV удален)
        excel_filename = f'{OUTPUT_DIR}/homeconcept_sofas_{timestamp}.xlsx'
        df_clean.to_excel(excel_filename, index=False)
        
        # Вывод результатов сбора данных
        print(f"\n" + "=" * 50)
        print("РЕЗУЛЬТАТЫ СБОРА ДАННЫХ")
        print("=" * 50)
        print(f"Всего товаров: {len(df_clean)}")
        print(f"Товаров со скидкой: {len(df_clean[df_clean['discount'] != 'Нет скидки'])}")
        
        # Статистика по ценам
        priced_products = df_clean[df_clean['price'] > 0]
        if len(priced_products) > 0:
            print(f"\nСТАТИСТИКА ПО ЦЕНАМ:")
            print(f"Товаров с ценами: {len(priced_products)}")
            print(f"Минимальная цена: {priced_products['price'].min():,} руб".replace(',', ' '))
            print(f"Максимальная цена: {priced_products['price'].max():,} руб".replace(',', ' '))
            print(f"Средняя цена: {priced_products['price'].mean():.0f} руб")
            print(f"Медианная цена: {priced_products['price'].median():.0f} руб")
        
        print(f"\nСОХРАНЕННЫЕ ФАЙЛЫ:")
        print(f"Excel: {excel_filename}")

        print("\nРезультат: Успешно собраны данные с 12 товаров, проведена очистка")
        print("и фильтрация, результаты сохранены в machine-readable форматах.")

        # Вывод таблицы с товарами
        print(f"\nСПИСОК ТОВАРОВ ({len(df_clean)}):")
        print("-" * 100)
        # Перебираем все товары и выводим их в форматированном виде
        for idx, row in df_clean.iterrows():
            # Форматируем цену с разделителями тысяч
            price_str = f"{row['price']:,} руб".replace(',', ' ') if row['price'] > 0 else "цена не указана"
            # Добавляем информацию о скидке если есть
            discount_str = f" ({row['discount']})" if row['discount'] != 'Нет скидки' else ""
            # Выводим товар в строку таблицы
            print(f"{idx+1:2d}. {row['title'][:60]:60} | {price_str:15}{discount_str}")
        
    else:
        print("Нет данных для сохранения")

# Точка входа в программу
if __name__ == "__main__":
    main()

ПАРСЕР HOMECONCEPT - ДИВАНЫ ДЛЯ УЛИЦЫ
Загрузка страницы: https://homeconcept.ru/catalog/outdoor/outdoor-sofas/
Найдено карточек товаров: 12
Обработка товара 1/12...
✓ Найден товар: Двухместный садовый диван Reef 2 Seater, Black акр... - 80500 руб
Обработка товара 2/12...
✓ Найден товар: Двухместный садовый диван Reef 2 Seater, Black акр... - 85200 руб
Обработка товара 3/12...
✓ Найден товар: Двухместный садовый диван Bluff 2 Seater, Black ак... - 111700 руб
Обработка товара 4/12...
✓ Найден товар: Двухместный садовый диван Bluff 2 Seater, Black ак... - 111700 руб
Обработка товара 5/12...
✓ Найден товар: Модульный садовый диван Blow Group акриловая ткань... - 181500 руб
Обработка товара 6/12...
✓ Найден товар: Трёхместный садовый диван Enjoy 3 Seater, Black ак... - 144900 руб
Обработка товара 7/12...
✓ Найден товар: Трёхместный садовый диван Enjoy 3 Seater, Black ак... - 144900 руб
Обработка товара 8/12...
✓ Найден товар: Трёхместный садовый диван Ethic 3 Seater, Black ак... - 112400 ру

C:\Users\kolba\AppData\Local\Temp\ipykernel_16732\1473514424.py:127: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  discount_elem = product_card.find(text=re.compile(r'Распродажа\s*\d+%'))


2. Учет местоположения: Сайт HomeConcept является интернет-магазином с фиксированными ценами по России, поэтому указание местоположения не требовалось.  
3. Анализ структуры URL: Проанализирована структура URL. В данной категории отсутствует пагинация - все товары отображаются на одной странице.   
4. Анализ кода страницы:   
* Определен ключевый элемент карточки товара: div class="item"
* Проанализирована HTML-структура каждого товара
* Выявлены селекторы для извлечения названия, цены, изображения и других данных   
5. Сбор данных: Написана программа, которая собирает следующие параметры:
* Название товара
* Цена
* Артикул (из URL)
* Ссылка на товар
* Изображение
* Информация о скидке
* Временная метка сбора данных
6. Проверка на корректность:
* Удаление дубликатов по названию и артикулу
* Фильтрация товаров без названия
* Проверка цен на корректность
7. Фильтрация по критерию: В программе предусмотрена возможность фильтрации:
* По наличию скидки
* По диапазону цен (может быть легко добавлено)
8. Сохранение для машинного анализа: Данные сохранены в формате Excel, который пригоден для дальнейшего машинного анализа.
9. Основные сложности парсинга:
+ Динамический контент - данные подгружаются через JavaScript
+ Защита от скрапинга - CAPTCHA, лимиты запросов, блокировки IP
+ Геоограничения - контент доступен только для определенных стран
+ Сложная структура - нестандартная кодировка, вложенные данные
*Главное ограничение:*
* Не все сайты доступны для парсинга. Многие ресурсы используют:
* обязательную авторизацию
* шифрование данных
* защиту типа Cloudflare
* клиентский рендеринг   
В таких случаях парсинг невозможен без официального API.